<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="cognitiveclass.ai logo">
</center>


# Machine Learning Foundation

## Course 4, Part e: Non-Negative Matrix Factorization DEMO


This exercise illustrates usage of Non-negative Matrix factorization and covers techniques related to sparse matrices and some basic work with Natural Langauge Processing.  We will use NMF to look at the top words for given topics.


## Data


We'll be using the BBC dataset. These are articles collected from 5 different topics, with the data pre-processed. 

These data are available in the data folder (or online [here](http://mlg.ucd.ie/files/datasets/bbc.zip?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01)). The data consists of a few files. The steps we'll be following are:

* *bbc.terms* is just a list of words 
* *bbc.docs* is a list of artcles listed by topic.

At a high level, we're going to 

1. Turn the `bbc.mtx` file into a sparse matrix (a [sparse matrix](https://docs.scipy.org/doc/scipy/reference/sparse.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01) format can be useful for matrices with many values that are 0, and save space by storing the position and values of non-zero elements).
1. Decompose that sparse matrix using NMF.
1. Use the resulting components of NMF to analyze the topics that result.


## Data Setup


Note: This lab has been updated to work in skillsnetwork for your convenience.


In [1]:
import urllib

In [2]:
with urllib.request.urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/bbc.mtx') as r:
    content = r.readlines()[2:]

## Part 1

Here, we will turn this into a list of tuples representing a [sparse matrix](https://docs.scipy.org/doc/scipy/reference/sparse.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01). Remember the description of the file from above:

* *bbc.mtx* is a list: first column is **wordID**, second is **articleID** and the third is the number of times that word appeared in that article.

So, if word 1 appears in article 3, 2 times, one element of our list will be:

`(1, 3, 2)`


In [6]:
content[0],content[1]

(b'1 1 1.0\n', b'1 7 2.0\n')

In [7]:
content[0].split()

[b'1', b'1', b'1.0']

In [12]:
tuple(map(float,content[0].split()))

(1.0, 1.0, 1.0)

In [14]:
tuple(map(int,map(float,content[0].split())))

(1, 1, 1)

In [3]:
sparsemat = [tuple(map(int,map(float,c.split()))) for c in content]
# Let's examine the first few elements
sparsemat[:8]

[(1, 1, 1),
 (1, 7, 2),
 (1, 11, 1),
 (1, 14, 1),
 (1, 15, 2),
 (1, 19, 2),
 (1, 21, 1),
 (1, 29, 1)]

## Part 2: Preparing Sparse Matrix data for NMF 


We will use the [coo matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01) function to turn the sparse matrix into an array. 


In [15]:
import numpy as np
from scipy.sparse import coo_matrix
rows = [x[0] for x in sparsemat]
cols = [x[1] for x in sparsemat]
values = [x[2] for x in sparsemat]
coo = coo_matrix((values, (rows, cols)))

## NMF


NMF is a way of decomposing a matrix of documents and words so that one of the matrices can be interpreted as the "loadings" or "weights" of each word on a topic. 


Check out [the NMF documentation](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01) and the [examples of topic extraction using NMF and LDA](http://scikit-learn.org/0.18/auto_examples/applications/topics_extraction_with_nmf_lda.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01).


## Part 3

Here, we will import `NMF`, define a model object with 5 components, and `fit_transform` the data created above.


In [20]:
# Surpress warnings from using older version of sklearn:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from sklearn.decomposition import NMF
model = NMF(n_components=5, init='random', random_state=818)
doc_topic = model.fit_transform(coo)

doc_topic.shape
# we should have 9636 observations (articles) and five latent features

(9636, 5)

In [22]:
doc_topic

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [2.43423348e+00, 7.01529551e-01, 1.03935248e+00, 5.39638492e-01,
        4.99462408e-01],
       [0.00000000e+00, 5.49866061e-01, 2.44845915e+00, 3.27169531e-01,
        2.97735273e-01],
       ...,
       [0.00000000e+00, 0.00000000e+00, 2.48510074e-03, 1.60218523e-03,
        2.10108039e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.72993384e-03,
        2.00619076e-02],
       [0.00000000e+00, 0.00000000e+00, 1.58662286e-03, 0.00000000e+00,
        1.46731272e-02]])

In [23]:
# find feature with highest value per doc
np.argmax(doc_topic, axis=1)

array([0, 0, 2, ..., 4, 4, 4])

In [24]:
np.argmax(doc_topic, axis=1).shape

(9636,)

## Part 4: 

Check out the `components` of this model:


In [26]:
model.components_

array([[0.00000000e+00, 0.00000000e+00, 1.38270552e-02, ...,
        1.16294930e-01, 2.67658884e-02, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 3.45973237e-03, 0.00000000e+00],
       [0.00000000e+00, 2.42318342e-01, 2.70555550e-01, ...,
        1.94836183e-01, 8.57914245e-02, 0.00000000e+00],
       [0.00000000e+00, 8.51673991e-03, 3.89478235e-03, ...,
        4.79348843e-02, 6.39591089e-03, 4.85501158e+00],
       [0.00000000e+00, 1.16747418e-01, 0.00000000e+00, ...,
        3.75339490e-01, 2.16110570e-01, 8.89277077e-02]])

In [25]:
model.components_.shape

(5, 2226)

This is five rows, each of which is a "topic" containing the weights of each word on that topic. The exercise is to _get a list of the top 10 words for each topic_. We can just store this in a list of lists.


**Note:** Just like we read in the data above, we'll have to read in the words from the `bbc.terms` file.


In [27]:
with urllib.request.urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/bbc.terms') as r:
    content = r.readlines()
words = [c.split()[0] for c in content]

In [31]:
content[:5]

[b'ad\n', b'sale\n', b'boost\n', b'time\n', b'warner\n']

In [32]:
words[:5]

[b'ad', b'sale', b'boost', b'time', b'warner']

In [35]:
topic_words = []
for r in model.components_:
    print(r)
    a = sorted([(v,i) for i,v in enumerate(r)],reverse=True)[0:12]
    topic_words.append([words[e[1]] for e in a])

[0.         0.         0.01382706 ... 0.11629493 0.02676589 0.        ]
[0.         0.         0.         ... 0.         0.00345973 0.        ]
[0.         0.24231834 0.27055555 ... 0.19483618 0.08579142 0.        ]
[0.00000000e+00 8.51673991e-03 3.89478235e-03 ... 4.79348843e-02
 6.39591089e-03 4.85501158e+00]
[0.         0.11674742 0.         ... 0.37533949 0.21611057 0.08892771]


In [36]:
# Here, each set of words relates to the corresponding topic (ie the first set of words relates to topic 'Business', etc.)
topic_words[:5]

[[b'bondi',
  b'stanlei',
  b'continent',
  b'mortgag',
  b'bare',
  b'least',
  b'extent',
  b'200',
  b'leav',
  b'frustrat',
  b'yuan',
  b'industri'],
 [b'manipul',
  b'teenag',
  b'drawn',
  b'go',
  b'prosecutor',
  b'herbert',
  b'host',
  b'protest',
  b'hike',
  b'nation',
  b'calcul',
  b'power'],
 [b'dimens',
  b'hous',
  b'march',
  b'wider',
  b'owner',
  b'intend',
  b'declin',
  b'forc',
  b'posit',
  b'founder',
  b'york',
  b'unavail'],
 [b'rome',
  b'ft',
  b'regain',
  b'lawmak',
  b'outright',
  b'resum',
  b'childhood',
  b'greatest',
  b'citi',
  b'stagnat',
  b'crown',
  b'bodi'],
 [b'build',
  b'empir',
  b'isol',
  b'\xc2\xa312',
  b'restructur',
  b'closer',
  b'plung',
  b'depreci',
  b'durham',
  b'race',
  b'juli',
  b'segreg']]

The original data had 5 topics, as listed in `bbc.docs` (which these topic words relate to). 

```
Business
Entertainment
Politics
Sport
Tech
```

In "real life", we would have found a way to use these to inform the model. But for this little demo, we can just compare the recovered topics to the original ones. And they seem to match reasonably well. The order is different, which is to be expected in this kind of model.


In [37]:
with urllib.request.urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/bbc.docs') as r:
    doc_content = r.readlines()
    
doc_content[:8]

[b'business.001\n',
 b'business.002\n',
 b'business.003\n',
 b'business.004\n',
 b'business.005\n',
 b'business.006\n',
 b'business.007\n',
 b'business.008\n']

---
### Machine Learning Foundation (C) 2020 IBM Corporation
